In [1]:
from dotenv import load_dotenv
load_dotenv()
import azure.functions as func
import os
import json
import math
import time
import datetime as dt
from datetime import datetime
import pytz
EST=pytz.timezone('US/Eastern')
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import deque
import pyodbc

In [3]:
tickers=['CVX','ABBV','KMX','JKHY']
conn_str='DRIVER={ODBC Driver 17 for SQL Server};SERVER='+os.environ.get('server')+ \
            ';DATABASE='+os.environ.get('database')+ \
                ';UID='+os.environ.get('dbusername')+ \
                    ';PWD='+ os.environ.get('dbpassword')
cnxn = pyodbc.connect(conn_str)       
cursor = cnxn.cursor()
query = "Select Distinct Ticker, DateIndex, High, Low, Open_, Close_, Volume From HourData Where DateIndex>DateAdd(Month,-6,GetDate()) and Ticker in ('{}')".format("','".join(tickers))
cursor.execute(query)
dataquery1 = [list(ele) for ele in cursor]
cnxn.close()
daily_df = pd.DataFrame(dataquery1,columns=['Ticker','Date','High','Low','Open','Close','Volume'])
daily_df.drop_duplicates(subset=['Date','Ticker'],inplace=True)
daily_df['Date']=pd.to_datetime(daily_df['Date'])
lastestdate = daily_df['Date'].max()
recenttickers=daily_df.loc[daily_df['Date']==lastestdate,'Ticker'].tolist()
daily_df=daily_df[daily_df['Ticker'].isin(recenttickers)]
daily_df['DateInt']=pd.to_numeric(daily_df['Date'].dt.strftime('%Y%m%d%H%M'))
daily_df=daily_df.pivot(index='Date',columns='Ticker',values=['High','Low','Open','Close','Volume','DateInt'])
daily_df.sort_index(inplace=True)
daily_df.interpolate(inplace=True)
# Get all tickers in the daily_df dataframe in the correct order
tickers=np.unique(daily_df['High'].columns.to_numpy(copy=True))
rows=len(daily_df.index)
#Create a 3D array for calculation of indicators
daily_np=np.empty(shape=(rows,6,tickers.shape[0]))
daily_df=daily_df.copy()
idx=pd.IndexSlice
#Build the numpy array by looping through the array and placing the array with the z-axis being ticker
for tn,t in enumerate(tickers):
    daily_np[:,:,tn]=daily_df.loc[:,idx[:,t]].to_numpy(copy=True)
np.nan_to_num(daily_np,copy=True)

y=daily_np.shape[0]
x=daily_np.shape[1]
z=daily_np.shape[2]

In [25]:
fisher=np.zeros(shape=(y,1,z))
r=y
period=50
days=800
Xax=np.arange(0,days,1)
for d in range(days,-1,-1):
    f=daily_np[r-period-d:r-d,3,:]
    # print(f.shape)
    minf=np.amin(f,0).flatten()
    maxf=np.amax(f,0).flatten()
    diff_f=maxf-minf
    scalarf=(f-minf)/diff_f*2-1
    scalarf=np.where(scalarf==-1,-.999,scalarf)
    scalarf=np.where(scalarf==1,.999,scalarf)
    for i in range(z):
        # print(i)
        fisher[-d,0,i]=.5*math.log(np.divide((1+scalarf[-1:,i]),(1-scalarf[-1:,i])))+.5*fisher[-d-1,0,i]
print(fisher[790:1400,:,3])

[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 1.091099  ]
 [ 1.74227491]
 [ 1.85928155]
 [ 4.72984194]
 [ 3.92913033]
 [ 3.29142892]
 [ 5.44591563]
 [ 5.00513191]
 [ 6.30276712]
 [ 4.67031849]
 [ 3.58807917]
 [ 2.79234649]
 [ 2.21176601]
 [ 1.93430643]
 [ 1.70617676]
 [ 0.96439899]
 [ 0.71314965]
 [ 0.53044767]
 [ 0.62535313]
 [ 0.65968551]
 [ 0.56079291]
 [ 0.44376128]
 [ 0.28406164]
 [ 0.12519812]
 [ 0.16921126]
 [ 0.11421381]
 [-0.07422925]
 [-0.08299706]
 [ 0.28800242]
 [ 0.51381078]
 [ 0.22156042]
 [-0.5803789 ]
 [-0.90516359]
 [-0.76135402]
 [-0.76996738]
 [-1.1469237 ]
 [-1.4905312 ]
 [-1.63469363]
 [-2.57262177]
 [-3.69846375]
 [-2.87674338]
 [-1.90351018]
 [-1.29982345]
 [-1.13486182]
 [-0.79855172]
 [-0.6430356 ]
 [-0.2094978 ]
 [ 0.18172315]
 [ 0.33885113]
 [ 0.3342487 ]
 [ 0.52419308]
 [ 1.30970924]
 [ 1.5648198 ]
 [ 1.54301674]
 [ 4.57170954]
 [ 6.08605594]
 [ 4.17306989]
 [ 2.93312426]
 [ 2.09192508]
 [ 1.66402